In [1]:
%pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118
Note: you may need to restart the kernel to use updated packages.


In [1]:
# All imports used in this project
import praw
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re
from sklearn.model_selection import train_test_split
import numpy as np
from torchmetrics import Accuracy, F1Score
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from tqdm import tqdm

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# Initialize the Reddit client
reddit = praw.Reddit(
    client_id='u6EMlVDn7jAwSKDA1A17Ww',
    client_secret='X2HbDELH7VjM6yOkRv9HXMJf1VcQXQ',
    user_agent='sentiment_app'
)

# Creating a list for the subreddit names and for storing the data
subreddit_names = ['scarystories', 'angry', 'funnystories', 'sadstories']
fear_data = [] 
anger_data = []
joy_data = []
sadness_data = []

# Storing and labelling data for each subreddit
for name in subreddit_names:
    subreddit = reddit.subreddit(name)  
    posts = subreddit.hot(limit=500)  
    for post in posts:
        post_words = post.selftext.split()
        if len(post_words) in range(120, 600):
            if name == 'scarystories':
                fear_data.append({'text': post.selftext, 'label': 'fear'})
            elif name == 'angry':
                anger_data.append({'text': post.selftext, 'label': 'anger'})
            elif name == 'funnystories':
                joy_data.append({'text': post.selftext, 'label': 'joy'})
            elif name == 'sadstories':
                sadness_data.append({'text': post.selftext, 'label': 'sadness'})

# Printing first 5 rows of data 
for entry in fear_data[:5]:
    print(entry)
    print("-----------------------------------------------------------------------------------------------------------------------")
for entry in anger_data[:5]:
    print(entry)
    print("-----------------------------------------------------------------------------------------------------------------------")
for entry in joy_data[:5]:
    print(entry)
    print("-----------------------------------------------------------------------------------------------------------------------")
for entry in sadness_data[:5]:
    print(entry)
    print("-----------------------------------------------------------------------------------------------------------------------")

# Saving all the data in one list    
all_data = fear_data + anger_data + joy_data + sadness_data

{'text': 'A few months ago, I moved into a new apartment. It was small, cheap, and perfect for my budget. The only weird thing? The apartment next to mine, 3B, always had this faint smell of something rotting. I figured it was just old food or maybe garbage that hadn’t been taken out, so I ignored it.\n\nOne night, around 2 a.m., I heard a faint tapping on my wall. I thought it was just the pipes, but then it started to sound like a soft, deliberate knocking. It was slow, rhythmic, and it didn’t stop. I knocked back, annoyed, and the tapping immediately stopped.\n\nThe next morning, I ran into my landlord in the hallway and casually asked about the neighbor in 3B. He looked at me with confusion and said, “Nobody’s lived in 3B for months. The last tenant disappeared without a trace, and no one’s moved in since.”\n\nThat night, I couldn’t sleep. Around 2 a.m., the tapping started again, but this time, it was louder and more urgent, like someone was pounding from inside the wall. I presse

In [3]:
# Printing the total number of data extracted from each subreddit
print(len(fear_data))
print(len(sadness_data))
print(len(joy_data))
print(len(anger_data))
print(len(all_data))

167
266
250
204
887


In [4]:
label_map = {"joy": 0, "anger": 1, "fear": 2, "sadness": 3}
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()  

# Function to preprocess the sentences
def preprocess_sentences(data):
    processed_sentences = []
    processed_data = []
    for item in data:
        sentence = item['text'].lower() # Convert all characters into lower case
        # Replace any new line characters with a space
        sentence = sentence.replace('\n', ' ').replace('\r', ' ').replace('\xa0\n\n', ' ') 
        # Remove any special charaters other than the ones listed
        sentence = re.sub(r'[^\w\s,:\'\"()?.,!;-{}\*_]', '', sentence)
        tokens = word_tokenize(sentence) # Tokenizes the sentence
        tokens = [token for token in tokens if token not in stop_words] # Removes stop words
        tokens = [stemmer.stem(token) for token in tokens] # Stems all the words
        processed_sentence = ' '.join(tokens) # Joins tokens to form the final sentence
        processed_sentences.append(processed_sentence) # Adds the final sentence to the list
        label = label_map[item['label']]
        processed_data.append({"text": processed_sentence, "label": label})
    return processed_data

# Function to encode the sentences using Tf-idfVectorizer
def encode_sentences(data):
    sentences = [item['text'] for item in data]
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(sentences)
    labels = [item['label'] for item in data]
    return X.toarray().astype(np.float32), labels, vectorizer

# Class to create our custom dataset
class EmotionDataset(Dataset):
    def __init__(self, x_data, y_data):
        self.x_data = torch.tensor(x_data, dtype=torch.float32)  # Convert features to tensors
        self.y_data = torch.tensor(y_data, dtype=torch.long)  # Convert labels to tensors
    def __len__(self):
        return len(self.x_data) # Returns the length of the data
    def __getitem__(self, idx):
        text = self.x_data[idx]
        label = self.y_data[idx]
        return {"text": text, "label": label} # Returns the text and it's label

# Function to implement the text processing pipeline
def text_processing_pipeline(data, test_size=0.2, batch_size=16):
    processed_data = preprocess_sentences(data) # Preprocesses the texts
    encoded_sentences, labels, vectorizer = encode_sentences(processed_data) # Encodes it
    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(encoded_sentences, labels, test_size=test_size, random_state=42)
    # Create datasets
    train_dataset = EmotionDataset(X_train, y_train)
    test_dataset = EmotionDataset(X_test, y_test)
    # Create dataloaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)
    return train_loader, test_loader, vectorizer

train_loader, test_loader, vectorizer = text_processing_pipeline(all_data)

In [5]:
# My own model
class EmotionRecognitionModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_layers, num_classes, dropout_prob=0.5):
        super(EmotionRecognitionModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.dropout_prob = dropout_prob
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        # Bidirectional LSTM
        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers, batch_first=True, bidirectional=True)
        # Dropout layer
        self.dropout = nn.Dropout(p=self.dropout_prob)
        # Layer normalization
        self.layer_norm = nn.LayerNorm(hidden_size * 2)  # Hidden size * 2 for bidirectional
        # Fully connected layer
        self.fc = nn.Linear(hidden_size * 2, num_classes)  # Output size is hidden_size * 2 for bidirectional

    def forward(self, x):
        # x contains word indices, we first convert them to embeddings
        x = self.embedding(x)
        h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(x.device)  # *2 for bidirectional
        c0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(x.device)  # *2 for bidirectional
        # Forward pass through LSTM
        out, _ = self.lstm(x, (h0, c0))
        # Take the output from the last time step
        out = out[:, -1, :]
        # Apply dropout
        out = self.dropout(out)
        # Apply layer normalization
        out = self.layer_norm(out)
        # Fully connected layer
        out = self.fc(out)
        return out

In [17]:
# Initializing important variables that will be used for training and testing the model

vocab_size = len(vectorizer.get_feature_names_out())
embedding_dim = 300
hidden_size = 256
num_layers = 2
num_classes = 4

model = EmotionRecognitionModel(vocab_size, embedding_dim, hidden_size, num_layers, num_classes) # Initializing the model
model.to('cuda') # Sending it to the GPU
criterion = nn.CrossEntropyLoss() # Initializing the loss function
optimizer = AdamW(model.parameters(), lr=1e-5) # Initializing the optimizer

accuracy = Accuracy(task="multiclass", num_classes=num_classes).to('cuda') # Initializing the accuracy metric
f1 = F1Score(task="multiclass", num_classes=num_classes).to('cuda') # Initializing the F1-score metric

In [18]:
# Training function
def train_model(model, train_loader, criterion, optimizer):
    model.train() # Putting the model in training mode
    for epoch in range(10):
        running_loss = 0.0
        progress_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch+1}/10")
        for i, batch in progress_bar:
            texts = batch['text'].long().to('cuda')
            labels = batch['label'].long().to('cuda')
            optimizer.zero_grad()  # Resetting gradients          
            outputs = model(texts) # Forward pass to get the output probabilities
            loss = criterion(outputs, labels)  # Calculating loss
            loss.backward()  # Back propagation
            optimizer.step()  # Performing an optimizer step
            running_loss += loss.item() * texts.size(0)  # Calculating the running loss      
        epoch_loss = running_loss / len(train_loader.dataset)  # Calculating the total loss for each epoch
        print(f'Epoch {(epoch+1)}/10, Loss: {epoch_loss:.4f}')
    print('Finished Training')

# Evaluation function
def evaluate_model(model, test_loader):
    model.eval() # Putting the model in evaluation mode
    all_labels = []
    all_predictions = []
    progress_bar = tqdm(test_loader, desc="Evaluating")
    with torch.no_grad():
        for batch in progress_bar:
            texts = batch['text'].long().to('cuda')
            labels = batch['label'].long().to('cuda')       
            outputs = model(texts)  # Forward pass to get the output probabilities
            _, predicted = torch.max(outputs, 1)  # Getting the class with maximum probability           
            # Moving all the labels and predictions to the CPU 
            # as NumPy cannot directly access them if they're in the gpu
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())
            # Calculating the accuracy and f1 score
            accuracy.update(predicted, labels)
            f1.update(predicted, labels)
    accuracy_value = accuracy.compute()
    f1_value = f1.compute()
    print(f'Test Accuracy: {accuracy_value:.4f}')
    print(f'Test F1 Score: {f1_value:.4f}')

    accuracy.reset()
    f1.reset()
    
# Run training and evaluation
train_model(model, train_loader, criterion, optimizer)
evaluate_model(model, test_loader)

Epoch 1/10: 100%|██████████| 45/45 [01:59<00:00,  2.66s/it]


Epoch 1/10, Loss: 1.4407


Epoch 2/10: 100%|██████████| 45/45 [01:59<00:00,  2.65s/it]


Epoch 2/10, Loss: 1.4526


Epoch 3/10: 100%|██████████| 45/45 [02:00<00:00,  2.68s/it]


...

Epoch 9/10, Loss: 1.4274


Epoch 10/10: 100%|██████████| 45/45 [01:59<00:00,  2.67s/it]


Epoch 10/10, Loss: 1.4265
Finished Training


Evaluating: 100%|██████████| 12/12 [00:12<00:00,  1.07s/it]

Test Accuracy: 0.2697
Test F1 Score: 0.2697


In [9]:
# Data Preprocessing for the DistilBERT Model

label_map = {"joy": 0, "anger": 1, "fear": 2, "sadness": 3}

# Preprocessing function (to only lowercasing and special character removal)
def preprocess_sentences(data):
    processed_data = []
    for item in data:
        sentence = item['text'].lower()
        sentence = sentence.replace('\n', ' ').replace('\r', ' ').replace('\xa0\n\n', ' ')
        sentence = re.sub(r'[^\w\s,:\'\"()?.,!;-{}\*_]', '', sentence)
        processed_data.append({"text": sentence, "label": label_map[item['label']]})
    return processed_data

# Dataset class for DistilBERT
class EmotionDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=128):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __len__(self):
        return len(self.data) 
    
    def __getitem__(self, idx):
        text = self.data[idx]['text']
        label = self.data[idx]['label']
        encoding = self.tokenizer.encode_plus(
            text,
            truncation=True,
            max_length=self.max_length,
            padding='max_length',
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'label': torch.tensor(label, dtype=torch.long)
        }

# Text processing pipeline 
def text_processing_pipeline(data, test_size=0.2, batch_size=16):
    processed_data = preprocess_sentences(data) # Preprocesses sentences
    # Splitting the data into training and testing sets
    train_data, test_data = train_test_split(processed_data, test_size=test_size, random_state=42)
    # Instantiating the DistilBERT tokenizer
    tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
    # Instantiating the training and testing data for the DistilBERT model
    train_dataset = EmotionDataset(train_data, tokenizer)
    test_dataset = EmotionDataset(test_data, tokenizer)
    # Instantiating the dataloader
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)
    
    return train_loader, test_loader

# Instantiating the training and testing data for the DistilBERT model
train_loader, test_loader = text_processing_pipeline(all_data)

c:\Users\ACER\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [12]:
# Instantiating the model
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=4)
model.to('cuda') # Moving the model to the GPU

optimizer = AdamW(model.parameters(), lr=1e-5) # Instantiating the optimizer

accuracy_metric = Accuracy(task="multiclass", num_classes=4).to('cuda') # Instantiating the accuracy metric
f1_metric = F1Score(task="multiclass", num_classes=4, average='weighted').to('cuda') # Instantiating the f1 metric

num_epochs = 10
for epoch in range(num_epochs):
    model.train() # Setting the model to training mode
    total_loss = 0
    for batch in tqdm(train_loader):
        input_ids = batch['input_ids'].to('cuda')
        attention_mask = batch['attention_mask'].to('cuda')
        labels = batch['label'].to('cuda')
        optimizer.zero_grad() # Resetting gradients
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels) # Forward pass to get the output probabilities
        loss = outputs.loss # Calculating loss
        total_loss += loss.item() # Calculating the running loss
        loss.backward() # Back propagation
        optimizer.step() # Performing an optimizer step
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(train_loader)}")

# Save the trained model
torch.save(model.state_dict(), 'distilbert_emotion_detection.pth')

model.eval() # Seting the model to evaluation mode
accuracy_metric.reset() # Resetting the accuracy metric
f1_metric.reset() # Resetting the f1 metric

# Evaluate the model
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to('cuda')
        attention_mask = batch['attention_mask'].to('cuda')
        labels = batch['label'].to('cuda')
        outputs = model(input_ids=input_ids, attention_mask=attention_mask) # Forward pass to get the output probabilities
        preds = torch.argmax(outputs.logits, dim=1) # Getting the class with the highest probability
        # Updating the metrics
        accuracy_metric.update(preds, labels)
        f1_metric.update(preds, labels)

# Compute final scores
accuracy = accuracy_metric.compute().item()
f1 = f1_metric.compute().item()

print(f"Accuracy: {accuracy:.4f}")
print(f"F1-Score: {f1:.4f}")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 46/46 [00:06<00:00,  7.34it/s]


Epoch 1/10, Loss: 1.3461448653884556


100%|██████████| 46/46 [00:05<00:00,  7.77it/s]


Epoch 2/10, Loss: 1.1806268018224966


100%|██████████| 46/46 [00:05<00:00,  7.75it/s]


...

Epoch 8/10, Loss: 0.16475849566252335


100%|██████████| 46/46 [00:05<00:00,  7.91it/s]


Epoch 9/10, Loss: 0.10784011706709862


100%|██████████| 46/46 [00:05<00:00,  7.87it/s]


Epoch 10/10, Loss: 0.08221789514241011
Accuracy: 0.7514
F1-Score: 0.7499


In [13]:
# Text preprocessing for the RoBERTa model
label_map = {"joy": 0, "anger": 1, "fear": 2, "sadness": 3}

# Preprocessing function (lowercasing and special character removal)
def preprocess_sentences(data):
    processed_data = []
    for item in data:
        sentence = item['text'].lower()
        sentence = sentence.replace('\n', ' ').replace('\r', ' ').replace('\xa0\n\n', ' ')
        sentence = re.sub(r'[^\w\s,:\'\"()?.,!;-{}\*_]', '', sentence)
        processed_data.append({"text": sentence, "label": label_map[item['label']]})
    return processed_data

# Dataset class for RoBERTa
class EmotionDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=128):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        text = self.data[idx]['text']
        label = self.data[idx]['label']
        encoding = self.tokenizer.encode_plus(
            text,
            truncation=True,
            max_length=self.max_length,
            padding='max_length',
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'label': torch.tensor(label, dtype=torch.long)
        }

# Text processing pipeline 
def text_processing_pipeline(data, test_size=0.2, batch_size=16):
    processed_data = preprocess_sentences(data) # Preprocessing the data
    # Splitting the data into training and testing sets
    train_data, test_data = train_test_split(processed_data, test_size=test_size, random_state=42)
    # Instantiating the DistilBERT tokenizer
    tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
    # Instantiating the training and testing datasets
    train_dataset = EmotionDataset(train_data, tokenizer)
    test_dataset = EmotionDataset(test_data, tokenizer)
    # Instantiating the training and testing dataloader
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)
    return train_loader, test_loader

# Instantiate and train RoBERTa model
train_loader, test_loader = text_processing_pipeline(all_data)


In [14]:
# Instantiating the RoBERTa model
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=4) 
model.to('cuda') # Moving the model to the GPU

optimizer = AdamW(model.parameters(), lr=1e-5) # Instantiating the optimizer

accuracy_metric = Accuracy(task="multiclass", num_classes=4).to('cuda') # Instantiating the accuracy metric
f1_metric = F1Score(task="multiclass", num_classes=4, average='weighted').to('cuda') # Instantiating the f1 metric

num_epochs = 10
for epoch in range(num_epochs):
    model.train() # Setting the model to training mode
    total_loss = 0
    for batch in tqdm(train_loader):
        input_ids = batch['input_ids'].to('cuda')
        attention_mask = batch['attention_mask'].to('cuda')
        labels = batch['label'].to('cuda')
        optimizer.zero_grad() # Resetting gradients
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels) # Forward pass to get the output probabilities
        loss = outputs.loss # Calculating the loss
        total_loss += loss.item() # Calculating the total loss
        loss.backward() # Back propagation
        optimizer.step() # Performing an optimizer step
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(train_loader)}")

# Save the trained model
torch.save(model.state_dict(), 'roberta_emotion_detection.pth')

model.eval() # Setting the model to evaluation mode
# Resetting the accuracy and f1 metrics
accuracy_metric.reset() 
f1_metric.reset()

# Evaluate the model
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to('cuda')
        attention_mask = batch['attention_mask'].to('cuda')
        labels = batch['label'].to('cuda')
        outputs = model(input_ids=input_ids, attention_mask=attention_mask) # Forward pass to get the output probabilities
        preds = torch.argmax(outputs.logits, dim=1) # Getting the class with the highest probability            
        # Update metrics
        accuracy_metric.update(preds, labels)
        f1_metric.update(preds, labels)

# Compute final scores
accuracy = accuracy_metric.compute().item()
f1 = f1_metric.compute().item()

print(f"Accuracy: {accuracy:.4f}")
print(f"F1-Score: {f1:.4f}")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 46/46 [00:11<00:00,  3.88it/s]


Epoch 1/10, Loss: 1.37367684426515


100%|██████████| 46/46 [00:11<00:00,  4.04it/s]


Epoch 2/10, Loss: 1.0513750211052273


100%|██████████| 46/46 [00:11<00:00,  4.11it/s]


...

Epoch 8/10, Loss: 0.07444361953631691


100%|██████████| 46/46 [00:11<00:00,  4.09it/s]


Epoch 9/10, Loss: 0.055243108502548675


100%|██████████| 46/46 [00:11<00:00,  4.00it/s]


Epoch 10/10, Loss: 0.03646675827305602
Accuracy: 0.8066
F1-Score: 0.8064
